In [1]:

from misc.visualizer import *

import numpy as np
import math
import time
import logging
from data.data_loader import Dataset
from misc.preferences import PREFERENCES
from misc.run_configuration import get_default_params, OutputLayerType, LearningSchedulerType, OptimizerType, hyperOpt_goodParams, default_params
from misc import utils

from optimizer import get_optimizer
from criterion import NllLoss, LossCombiner

from models.transformer.encoder import TransformerEncoder
from models.jointAspectTagger import JointAspectTagger
from trainer.train import Trainer
import pprint
from data.germeval2017 import germeval2017_dataset as dsl

import torch
import pprint

import torchtext
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup

In [7]:
def load_model(dataset, rc, experiment_name):
    loss = LossCombiner(4, dataset.class_weights, NllLoss)
    transformer = TransformerEncoder(dataset.source_embedding,
                                     hyperparameters=rc)
    model = JointAspectTagger(transformer, rc, 4, 20, dataset.target_names)
    optimizer = get_optimizer(model, rc)
    trainer = Trainer(
                        model,
                        loss,
                        optimizer,
                        rc,
                        dataset,
                        experiment_name,
                        enable_tensorboard=False,
                        verbose=False)
    return trainer

def load_dataset(rc, logger, task):
    dataset = Dataset(
        task,
        logger,
        rc,
        source_index=PREFERENCES.source_index,
        target_vocab_index=PREFERENCES.target_vocab_index,
        data_path=PREFERENCES.data_root,
        train_file=PREFERENCES.data_train,
        valid_file=PREFERENCES.data_validation,
        test_file=PREFERENCES.data_test,
        file_format=PREFERENCES.file_format,
        init_token=None,
        eos_token=None
    )
    dataset.load_data(dsl, verbose=False)
    return dataset

In [8]:
PREFERENCES.defaults(
    data_root='./data/data/germeval2017',
    data_train='train_v1.4.tsv',    
    data_validation='dev_v1.4.tsv',
    data_test='test_TIMESTAMP1.tsv',
    source_index=0,
    target_vocab_index=2,
    file_format='csv'
)
main_experiment_name = 'GermEval7_Experiments'
use_cuda = True

In [9]:
baseline = {**default_params, **hyperOpt_goodParams}
print(pprint.pformat(baseline, indent=2))
utils.get_current_git_commit()
print('Current commit: ' + utils.get_current_git_commit())

{ 'att_d_k': 300,
  'att_d_v': 300,
  'batch_size': 12,
  'clip_comments_to': 113,
  'dropout_rate': 0.302424,
  'early_stopping': 5,
  'embedding_dim': 300,
  'embedding_name': '6B',
  'embedding_type': 'fasttext',
  'harmonize_bahn': True,
  'language': 'de',
  'learning_rate_scheduler': { 'noam_learning_rate_factor': 1.418,
                               'noam_learning_rate_warmup': 8000},
  'learning_rate_scheduler_type': <LearningSchedulerType.Noam: 1>,
  'log_every_xth_iteration': -1,
  'model_size': 300,
  'num_encoder_blocks': 2,
  'num_epochs': 25,
  'num_heads': 1,
  'optimizer': { 'adam_beta1': 0.81,
                 'adam_beta2': 0.7173,
                 'adam_eps': 0.000814,
                 'learning_rate': 7.2e-05},
  'optimizer_type': <OptimizerType.Adam: 1>,
  'organic_text_cleaning': False,
  'output_dropout_rate': 0.79602089766246,
  'output_layer': { 'output_conv_kernel_size': 5,
                    'output_conv_num_filters': 300,
                    'output_conv_pa

In [10]:
experiment_name = utils.create_loggers(experiment_name=main_experiment_name)
logger = logging.getLogger(__name__)
dataset_logger = logging.getLogger('data_loader')
rc = get_default_params(use_cuda=True, overwrite={}, from_default=baseline)

dataset = load_dataset(rc, dataset_logger, rc.task)
logger.debug('dataset loaded')


Log path is  C:\Users\felix\OneDrive\Studium\Studium\6. Semester\MA\Project\ABSA-Transformer\logs\GermEval7_Experiments\20190411\1


In [11]:
logger.debug('Load model')
trainer.load_model(custom_path='C:\\Users\\felix\\OneDrive\\Studium\\Studium\\6. Semester\\MA\\Project\\ABSA-Transformer\\logs\\GermEval7_Experiments\\20190401\\0\\checkpoints')
trainer.set_cuda(True)
result = trainer.perform_final_evaluation(use_test_set=True, verbose=False)
print(result)

NameError: name 'trainer' is not defined

In [ ]:
def produce_test_gold_labels(iterator: torchtext.data.Iterator, dataset: Dataset, filename='evaluation_output.xml'):

    fields = dataset.fields
    with torch.no_grad():
        iterator.init_epoch()
        
        tree = ET.ElementTree()
        root = ET.Element('Documents')

        for batch in iterator:
            doc_id, comment, relevance, aspect_sentiment, general_sentiment = batch.id, batch.comments, batch.relevance, batch.aspect_sentiments, batch.general_sentiments
            doc_id = fields['id'].reverse(doc_id.unsqueeze(1))
            comment = fields['comments'].reverse(comment)
            relevance = ['false' if r == 0 else 'true' for r in relevance]
            general_sentiment = fields['general_sentiments'].reverse(general_sentiment.unsqueeze(1))
            aspect_sentiment = fields['aspect_sentiments'].reverse(aspect_sentiment, detokenize=False)

            for i in range(len(doc_id)):
                docuement_elem = ET.SubElement(root, 'Document', {'id': doc_id[i]})

                rel_field = ET.SubElement(docuement_elem, 'relevance')
                rel_field.text = relevance[i]

                sen_field = ET.SubElement(docuement_elem, 'sentiment')
                sen_field.text = general_sentiment[i]

                text_field = ET.SubElement(docuement_elem, 'text')
                text_field.text = comment[i]

                # don't add aspects if field not relevant
                if relevance[i] == 'false':
                    continue
                options_elem = ET.SubElement(docuement_elem, 'Opinions')

                # add aspects
                for sentiment, a_name in zip(aspect_sentiment[i], dataset.target_names):
                    if sentiment == 'n/a':
                        continue

                    asp_field = ET.SubElement(options_elem, 'Opinion', {
                        'category': a_name,
                        'polarity': sentiment
                    })

        #print(BeautifulSoup(ET.tostring(tree), "xml").prettify())
        tree._setroot(root)
        tree.write(filename, encoding='utf-8')


In [ ]:
produce_test_gold_labels(dataset.test_iter, dataset)